In [1]:
import pandas as pd
import plotly.graph_objects as go
import datetime
import numpy as np
from copy import deepcopy
import cProfile
import sys
from collections import defaultdict
from tqdm import tqdm 
import pickle
from bisect import bisect_left
import random
from tqdm import tqdm
import random
import logging


from graph import TransportGraph, ContactionTransportGraph
from ttf import TTF
from forward_search import FCH
from dijkstra import Dijkstra

# Build transport graph

In [2]:
CITY = 'kuopio'

## Build Transport Graph

In [3]:
transport_connections = pd.read_csv(F'data/{CITY}/network_temporal_day.csv', sep=';')
walk_connections = pd.read_csv(F'data/{CITY}/network_walk.csv', sep=';')

In [4]:
df_walk_invert = walk_connections.copy()
df_walk_invert = df_walk_invert.rename(columns={'from_stop_I': 'to_stop_I', 'to_stop_I': 'from_stop_I'})
walk_connections = pd.concat((walk_connections, df_walk_invert))

In [5]:
tg = TransportGraph(transport_connections=transport_connections, walk_connections=walk_connections)

# Convert ATF to TTF

In [6]:
tg_ttf = deepcopy(tg)
for node1, out in tg_ttf.graph.items():
    for node2, f in out.items():
        tg_ttf.graph[node1][node2] = TTF(f)

## Graph statistics

In [6]:
tg.edges_cnt, tg.nodes_cnt, tg.timetable_stats

### Save TG with ATF and TG with TTF

In [ ]:
pickle.dump(tg, open(F'{CITY}.pkl', 'wb'), 
            pickle.HIGHEST_PROTOCOL)
pickle.dump(tg_ttf, open(F'{CITY}_ttf.pkl', 'wb'), 
            pickle.HIGHEST_PROTOCOL)

# Build CH graph

In [6]:
%%time
cProfile.run('ch_tg = tg.contraction_hierarchy()')

## Graph statistics

In [7]:
ch_tg.edges_cnt, ch_tg.nodes_cnt, ch_tg.timetable_stats

### Calculate Geometrical Containers for speed up future search

In [8]:
ch_tg.geometrical_container()

# Convert ATF to TTF in CH-graph

In [ ]:
ch_tg_ttf = deepcopy(ch_tg)
for node1, out in ch_tg_ttf.graph.items():
    for node2, f in out.items():
        ch_tg_ttf.graph[node1][node2] = TTF(f)

## Save graph

In [ ]:
pickle.dump(ch_tg, open(F'{CITY}_ch.pkl', 'wb'), 
            pickle.HIGHEST_PROTOCOL)
pickle.dump(ch_tg, open(F'{CITY}_ch_ttf.pkl', 'wb'), 
            pickle.HIGHEST_PROTOCOL)

# Calulate TTN for TG AND CH-graph

In [14]:
pickle.dump(ch_tg, open(F'graph/ch_{CITY}_fractional_new.pkl', 'wb'), 
            pickle.HIGHEST_PROTOCOL)

In [61]:
ch_tg.position_in_edge = defaultdict(dict)
ch_tg.nodes_schedule = defaultdict(dict)

In [62]:
ch_tg.m_arr_fractional = {}
ch_tg.pointers = {}
ch_tg.reachable_nodes = {}

In [9]:
tg.optimize_binary_search()

In [10]:
tg.fractional_cascading_precomputation()

In [11]:
ch_tg.optimize_binary_search()

In [12]:
ch_tg.fractional_cascading_precomputation()

# Compare solutions

In [14]:
N = 1000
test_data = pd.DataFrame({'start_time': [random.randint(transport_connections['dep_time_ut'].min(), 
                                           transport_connections['dep_time_ut'].max()) for i in range(N)],
             'start_node' : [random.sample(tg.nodes, 1)[0] for i in range(N)], 
              'end_node' : [random.sample(tg.nodes, 1)[0] for i in range(N)]
             })

algorithms = ['fch', 'fch_fractional', 'fch_binary_duration', 'dijkstra', 'dijkstra_binary_duration', 'dijkstra_fractional']
duration = {x: [] for x in algorithms}
arrival = {x: 0 for x in algorithms}
for index, row in test_data.iterrows():
    random.shuffle(algorithms)
    for algorithm in algorithms:
        if algorithm == 'fch':
            pathfinding = FCH(graph=ch_tg,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
            path = pathfinding.shortest_path(60, optimized_binary_search=False, 
                                             next_index_optimization=False)
            if path['path']:
                duration[algorithm].append(path['duration'])
            arrival[algorithm] = path['arrival']
        elif algorithm == 'fch_fractional':
            pathfinding = FCH(graph=ch_tg,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
            path = pathfinding.shortest_path(60, optimized_binary_search=True, 
                                             fractional_cascading=True)
            if path['path']:
                duration[algorithm].append(path['duration'])
            arrival[algorithm] = path['arrival']
        elif algorithm == 'fch_binary_duration':
            pathfinding = FCH(graph=ch_tg,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
            path = pathfinding.shortest_path(60, optimized_binary_search=True,
                                             next_index_optimization=False)
            if path['path']:
                duration[algorithm].append(path['duration'])
            arrival[algorithm] = path['arrival']
            
        elif algorithm == 'dijkstra':
            pathfinding = Dijkstra(graph=tg,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
            path = pathfinding.shortest_path(60, optimized_binary_search=False)
            if path['path']:
                duration[algorithm].append(path['duration'])
            arrival[algorithm] = path['arrival']
        elif algorithm == 'dijkstra_fractional':
            pathfinding = Dijkstra(graph=tg,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
            path = pathfinding.shortest_path(60, optimized_binary_search=True, fractional_cascading=True)
            if path['path']:
                duration[algorithm].append(path['duration'])
            arrival[algorithm] = path['arrival']
        elif algorithm == 'dijkstra_binary_duration':
            pathfinding = Dijkstra(graph=tg,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
            path = pathfinding.shortest_path(60, optimized_binary_search=True)
            if path['path']:
                duration[algorithm].append(path['duration'])
            arrival[algorithm] = path['arrival']
    arr = arrival[algorithms[0]]        
    for i in range(1, len(algorithms)):
        assert arr == arrival[algorithms[i]]  

In [15]:
for algorithm in algorithms:
    print(algorithm, np.mean(duration[algorithm]), np.median(duration[algorithm]), np.std(duration[algorithm]))

In [20]:
for algorithm in algorithms:
    print(algorithm, np.mean(duration[algorithm]), np.median(duration[algorithm]), np.std(duration[algorithm]))

In [18]:
for algorithm in algorithms:
    print(algorithm, np.mean(duration[algorithm]), np.median(duration[algorithm]), np.std(duration[algorithm]))

In [16]:
for algorithm in algorithms:
    print(algorithm, np.mean(duration[algorithm]), np.median(duration[algorithm]), np.std(duration[algorithm]))

In [ ]:
tg.graph[67]

In [31]:
len(ch_tg.m_arr_fractional.get(63)[0])

In [32]:
len(ch_tg.nodes_schedule.get(63))

In [26]:
len(tg.m_arr_fractional.get(67)[0])

In [18]:
data = [[{'algorithm': 'dijkstra',
   'average_outgoing_edges': 3.8921973484025214,
   'duration': 50.07529411764706}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.892414692458161,
   'duration': 39.18016194331984}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.9189306672462507,
   'duration': 21.63905325443787}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.2058248206911542,
   'duration': 7.95}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 0.7315800912845034,
   'duration': 3.4444444444444446}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 0.4379482721147577,
   'duration': 0.0}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 3.8921973484025214,
   'duration': 42.74}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.892414692458161,
   'duration': 31.910931174089068}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.9189306672462507,
   'duration': 18.5207100591716}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.2058248206911542,
   'duration': 6.75}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 0.7315800912845034,
   'duration': 2.7777777777777777}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 0.4379482721147577,
   'duration': 0.0}]]

In [19]:
# Define colors for each algorithm
colors = {
    'dijkstra': 'blue',
    'dijkstra_binary_duration': 'red'
}

fig = go.Figure()

for sublist in data:
    for item in sublist:
        algorithm = item['algorithm']
        avg_outgoing_edges = item['average_outgoing_edges']
        duration = item['duration']
        fig.add_trace(go.Scatter(x=[avg_outgoing_edges], y=[duration], mode='markers', marker=dict(color=colors[algorithm]), name=algorithm))

fig.update_layout(title='Algorithm Performance',
                  xaxis_title='Average Outgoing Edges',
                  yaxis_title='Duration')

fig.show()

In [26]:
data = [[{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 10.371584699453551,
   'duration': 7.732262382864793}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 7.111111111111111,
   'duration': 12.339479392624728}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 13.214936247723132,
   'duration': 9.299031476997579}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 3.9836065573770494,
   'duration': 8.369509043927648}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.0856102003642987,
   'duration': 7.720214190093708}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 16.194899817850636,
   'duration': 12.339479392624728}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.395264116575592,
   'duration': 6.824915824915825}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 0.9854280510018215,
   'duration': 7.9299719887955185}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 12.236794171220401,
   'duration': 9.70875}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 3.1020036429872495,
   'duration': 7.928571428571429}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 4.515482695810564,
   'duration': 8.87875}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.726775956284153,
   'duration': 7.467065868263473}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 8.666666666666666,
   'duration': 7.471556886227545}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 7.111111111111111,
   'duration': 11.436008676789587}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.6429872495446265,
   'duration': 10.95906432748538}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.46448087431694,
   'duration': 10.150121065375302}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.098360655737705,
   'duration': 11.425162689804772}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 3.9836065573770494,
   'duration': 9.142118863049095}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 15.19672131147541,
   'duration': 10.2439293598234}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 11.278688524590164,
   'duration': 9.130490956072352}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 7.870673952641166,
   'duration': 6.314814814814815}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 3.1020036429872495,
   'duration': 7.329131652661064}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 16.194899817850636,
   'duration': 11.42407809110629}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.2021857923497268,
   'duration': 9.126614987080103}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 6.38615664845173,
   'duration': 11.15121412803532}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 10.371584699453551,
   'duration': 8.437751004016064}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 3.5154826958105647,
   'duration': 8.44578313253012}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 5.080145719489982,
   'duration': 10.148910411622277}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 11.278688524590164,
   'duration': 8.37467700258398}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 7.870673952641166,
   'duration': 6.826599326599327}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 9.489981785063753,
   'duration': 7.310924369747899}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.3224043715846994,
   'duration': 8.8375}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.395264116575592,
   'duration': 6.318181818181818}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 15.19672131147541,
   'duration': 11.164459161147903}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.854280510018215,
   'duration': 10.219646799116997}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 13.214936247723132,
   'duration': 10.164648910411623}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 5.701275045537341,
   'duration': 10.971929824561403}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 9.489981785063753,
   'duration': 7.917366946778712}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 0.9854280510018215,
   'duration': 7.312324929971989}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 4.515482695810564,
   'duration': 9.69875}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.3224043715846994,
   'duration': 9.70875}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 3.5154826958105647,
   'duration': 7.720214190093708}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.0856102003642987,
   'duration': 8.452476572958501}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 5.080145719489982,
   'duration': 9.301452784503631}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 12.236794171220401,
   'duration': 8.8475}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 6.38615664845173,
   'duration': 10.237306843267108}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 8.666666666666666,
   'duration': 6.877245508982036}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.2021857923497268,
   'duration': 8.381136950904393}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 5.701275045537341,
   'duration': 10.03625730994152}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 14.202185792349727,
   'duration': 10.95906432748538}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.854280510018215,
   'duration': 11.147902869757175}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 14.202185792349727,
   'duration': 10.024561403508772}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.098360655737705,
   'duration': 12.366594360086768}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.726775956284153,
   'duration': 6.883233532934132}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.46448087431694,
   'duration': 9.297820823244551}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.6429872495446265,
   'duration': 10.024561403508772}]]

In [27]:
# Define colors for each algorithm
colors = {
    'dijkstra': 'blue',
    'dijkstra_binary_duration': 'red'
}

fig = go.Figure()

for sublist in data:
    for item in sublist:
        algorithm = item['algorithm']
        avg_outgoing_edges = item['average_outgoing_edges']
        duration = item['duration']
        fig.add_trace(go.Scatter(x=[avg_outgoing_edges], y=[duration], mode='markers', marker=dict(color=colors[algorithm]), name=algorithm))

fig.update_layout(title='Algorithm Performance',
                  xaxis_title='Average Outgoing Edges',
                  yaxis_title='Duration')

fig.show()

In [22]:
data = [[{'algorithm': 'dijkstra',
   'average_outgoing_edges': 3.8921973484025214}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.892414692458161}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.9189306672462507}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.2058248206911542}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 0.7315800912845034}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 0.4379482721147577}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 3.8921973484025214}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.892414692458161}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.9189306672462507}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.2058248206911542}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 0.7315800912845034}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 0.4379482721147577}]]
for d_ in data:
    d = d_[0]
    if d['algorithm']=='dijkstra':
        d['duration'] = d['average_outgoing_edges']*np.log(56)
    else:
        d['duration'] = d['average_outgoing_edges'] + np.log(d['average_outgoing_edges']) + np.log(56)

In [25]:
# Define colors for each algorithm
colors = {
    'dijkstra': 'blue',
    'dijkstra_binary_duration': 'red'
}

fig = go.Figure()

for sublist in data:
    for item in sublist:
        algorithm = item['algorithm']
        avg_outgoing_edges = item['average_outgoing_edges']
        duration = item['duration']
        fig.add_trace(go.Scatter(x=[avg_outgoing_edges], y=[duration], mode='markers', marker=dict(color=colors[algorithm]), name=algorithm))

fig.update_layout(title='Algorithm Performance',
                  xaxis_title='Average Outgoing Edges',
                  yaxis_title='Duration')

fig.show()

In [17]:
import plotly.graph_objects as go

# Sample data
x = [3.8921973484025214, 2.892414692458161, 1.9189306672462507, 1.2058248206911542, 0.7315800912845034]
y = [99.9, 38.3, 19.5, 9.6, 4.63]

# Create a trace
trace = go.Scatter(x=x, y=y, mode='markers')

# Create a figure and add the trace
fig = go.Figure(trace)

# Show the plot
fig.show()

In [28]:
data = [[{'algorithm': 'dijkstra',
   'average_outgoing_edges': 5.701275045537341,
   'duration': 7.263466042154567}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 11.278688524590164,
   'duration': 4.498002663115845}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 13.214936247723132,
   'duration': 6.871446229913474}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 14.202185792349727,
   'duration': 5.325526932084309}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 3.9836065573770494,
   'duration': 4.484687083888149}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 3.1020036429872495,
   'duration': 5.398843930635838}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 15.19672131147541,
   'duration': 5.48953488372093}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 14.202185792349727,
   'duration': 7.26463700234192}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 3.1020036429872495,
   'duration': 3.9927745664739884}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 12.236794171220401,
   'duration': 4.767352185089974}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.726775956284153,
   'duration': 3.836115326251897}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 7.870673952641166,
   'duration': 3.4725806451612904}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 10.371584699453551,
   'duration': 4.272851296043656}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 8.666666666666666,
   'duration': 3.8088012139605465}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 7.111111111111111,
   'duration': 5.855263157894737}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.0856102003642987,
   'duration': 4.286493860845839}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.3224043715846994,
   'duration': 6.596401028277635}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 7.870673952641166,
   'duration': 4.593548387096774}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 13.214936247723132,
   'duration': 4.932014833127318}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 16.194899817850636,
   'duration': 8.18421052631579}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 3.5154826958105647,
   'duration': 5.795361527967258}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.0856102003642987,
   'duration': 5.795361527967258}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 10.371584699453551,
   'duration': 5.796725784447476}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.2021857923497268,
   'duration': 6.292942743009321}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 9.489981785063753,
   'duration': 3.9739884393063583}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 4.515482695810564,
   'duration': 4.760925449871466}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 3.5154826958105647,
   'duration': 4.285129604365621}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 0.9854280510018215,
   'duration': 5.404624277456647}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 3.9836065573770494,
   'duration': 6.1171770972037285}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 16.194899817850636,
   'duration': 5.847587719298246}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 12.236794171220401,
   'duration': 6.592544987146529}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.2021857923497268,
   'duration': 6.11051930758988}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 6.38615664845173,
   'duration': 7.627906976744186}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.6429872495446265,
   'duration': 7.2716627634660425}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 9.489981785063753,
   'duration': 5.401734104046243}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.854280510018215,
   'duration': 7.638372093023256}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.726775956284153,
   'duration': 4.91350531107739}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 6.38615664845173,
   'duration': 5.472093023255814}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 4.515482695810564,
   'duration': 6.571979434447301}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.854280510018215,
   'duration': 5.467441860465116}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 0.9854280510018215,
   'duration': 3.995664739884393}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.395264116575592,
   'duration': 4.582258064516129}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.098360655737705,
   'duration': 8.179824561403509}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 11.278688524590164,
   'duration': 6.113182423435419}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.46448087431694,
   'duration': 6.860321384425216}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.395264116575592,
   'duration': 3.456451612903226}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 15.19672131147541,
   'duration': 7.636046511627907}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 5.701275045537341,
   'duration': 5.322014051522248}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.098360655737705,
   'duration': 5.843201754385965}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 7.111111111111111,
   'duration': 8.18311403508772}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 5.080145719489982,
   'duration': 4.930778739184178}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.3224043715846994,
   'duration': 4.76478149100257}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.6429872495446265,
   'duration': 5.327868852459017}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 8.666666666666666,
   'duration': 4.922610015174507}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.46448087431694,
   'duration': 4.953028430160693}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 5.080145719489982,
   'duration': 6.852904820766378}]]

In [29]:
# Define colors for each algorithm
colors = {
    'dijkstra': 'blue',
    'dijkstra_binary_duration': 'red'
}

fig = go.Figure()

for sublist in data:
    for item in sublist:
        algorithm = item['algorithm']
        avg_outgoing_edges = item['average_outgoing_edges']
        duration = item['duration']
        fig.add_trace(go.Scatter(x=[avg_outgoing_edges], y=[duration], mode='markers', marker=dict(color=colors[algorithm]), name=algorithm))

fig.update_layout(title='Algorithm Performance',
                  xaxis_title='Average Outgoing Edges',
                  yaxis_title='Duration')

fig.show()

In [30]:
data = [[{'algorithm': 'dijkstra',
   'average_outgoing_edges': 16.194899817850636,
   'duration': 8.183189655172415}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 10.48816029143898,
   'duration': 4.08852867830424}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 12.276867030965391,
   'duration': 4.595061728395062}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 14.21311475409836,
   'duration': 5.2734375}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.083788706739526,
   'duration': 7.788104089219331}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 8.868852459016393,
   'duration': 4.826322930800543}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 14.21311475409836,
   'duration': 7.328125}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 16.194899817850636,
   'duration': 5.807112068965517}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 5.5136612021857925,
   'duration': 6.8190255220417635}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 4.981785063752277,
   'duration': 4.597530864197531}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 5.5136612021857925,
   'duration': 4.909512761020881}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.3205828779599273,
   'duration': 6.8294663573085845}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 11.369763205828779,
   'duration': 5.895910780669145}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.8524590163934427,
   'duration': 8.181034482758621}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.2003642987249545,
   'duration': 6.345679012345679}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 3.096539162112933,
   'duration': 3.3314527503526095}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 4.100182149362477,
   'duration': 4.084788029925187}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 6.0783242258652095,
   'duration': 5.293526785714286}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 11.369763205828779,
   'duration': 4.291201982651796}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 10.48816029143898,
   'duration': 5.576059850374065}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 4.5136612021857925,
   'duration': 5.897149938042132}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.4626593806921675,
   'duration': 7.332589285714286}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 4.981785063752277,
   'duration': 6.334567901234568}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.2003642987249545,
   'duration': 4.5851851851851855}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 12.276867030965391,
   'duration': 6.346913580246913}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 3.3934426229508197,
   'duration': 3.5603799185888736}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.6411657559198543,
   'duration': 5.583603020496224}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 7.3843351548269585,
   'duration': 7.425646551724138}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.083788706739526,
   'duration': 4.302354399008674}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.8907103825136613,
   'duration': 3.757105943152455}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 13.234972677595628,
   'duration': 6.816705336426915}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 6.699453551912568,
   'duration': 7.797195253505933}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 6.0783242258652095,
   'duration': 7.330357142857143}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.4626593806921675,
   'duration': 5.277901785714286}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 8.868852459016393,
   'duration': 3.57123473541384}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 13.234972677595628,
   'duration': 4.916473317865429}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 3.096539162112933,
   'duration': 4.4724964739069115}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.9836065573770492,
   'duration': 4.087281795511222}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 3.724954462659381,
   'duration': 3.757105943152455}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 3.724954462659381,
   'duration': 5.103359173126615}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 4.5136612021857925,
   'duration': 4.302354399008674}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.8907103825136613,
   'duration': 5.109819121447028}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.6411657559198543,
   'duration': 7.793959007551241}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 6.699453551912568,
   'duration': 5.590075512405609}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.8524590163934427,
   'duration': 5.8125}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 9.664845173041895,
   'duration': 5.103359173126615}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 15.200364298724955,
   'duration': 5.586839266450917}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.9836065573770492,
   'duration': 5.562344139650873}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 9.664845173041895,
   'duration': 3.748062015503876}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.3205828779599273,
   'duration': 4.931554524361949}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 7.3843351548269585,
   'duration': 8.181034482758621}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 8.109289617486338,
   'duration': 4.4724964739069115}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 3.3934426229508197,
   'duration': 4.815468113975577}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 8.109289617486338,
   'duration': 5.327221438645981}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 4.100182149362477,
   'duration': 5.574812967581047}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 15.200364298724955,
   'duration': 7.788565264293419}]]

In [31]:
# Define colors for each algorithm
colors = {
    'dijkstra': 'blue',
    'dijkstra_binary_duration': 'red'
}

fig = go.Figure()

for sublist in data:
    for item in sublist:
        algorithm = item['algorithm']
        avg_outgoing_edges = item['average_outgoing_edges']
        duration = item['duration']
        fig.add_trace(go.Scatter(x=[avg_outgoing_edges], y=[duration], mode='markers', marker=dict(color=colors[algorithm]), name=algorithm))

fig.update_layout(title='Algorithm Performance',
                  xaxis_title='Average Outgoing Edges',
                  yaxis_title='Duration')

fig.show()

In [32]:
data=[[{'algorithm': 'dijkstra',
   'average_outgoing_edges': 9.664845173041895,
   'duration': 5.001}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 10.48816029143898,
   'duration': 3.915}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.6411657559198543,
   'duration': 5.456}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.4626593806921675,
   'duration': 5.031}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 15.200364298724955,
   'duration': 7.637}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 12.276867030965391,
   'duration': 4.349}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 8.868852459016393,
   'duration': 4.676}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 11.369763205828779,
   'duration': 5.623}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 8.109289617486338,
   'duration': 5.597}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 7.3843351548269585,
   'duration': 8.031}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.2003642987249545,
   'duration': 4.354}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 7.3843351548269585,
   'duration': 5.73}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.2003642987249545,
   'duration': 5.992}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 10.48816029143898,
   'duration': 5.323}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 3.3934426229508197,
   'duration': 5.989}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 13.234972677595628,
   'duration': 6.505}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.3205828779599273,
   'duration': 6.524}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.6411657559198543,
   'duration': 7.632}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.8907103825136613,
   'duration': 5.012}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 11.369763205828779,
   'duration': 4.147}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 3.096539162112933,
   'duration': 3.197}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.8524590163934427,
   'duration': 5.73}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 12.276867030965391,
   'duration': 5.982}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 16.194899817850636,
   'duration': 8.043}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 4.981785063752277,
   'duration': 5.989}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 3.724954462659381,
   'duration': 5.007}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.4626593806921675,
   'duration': 6.945}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 16.194899817850636,
   'duration': 5.722}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 8.868852459016393,
   'duration': 3.479}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.8524590163934427,
   'duration': 8.045}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 4.100182149362477,
   'duration': 5.339}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.9836065573770492,
   'duration': 5.348}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 4.5136612021857925,
   'duration': 5.609}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 6.699453551912568,
   'duration': 7.63}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 9.664845173041895,
   'duration': 3.73}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 6.0783242258652095,
   'duration': 6.955}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 3.096539162112933,
   'duration': 4.258}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 3.3934426229508197,
   'duration': 3.483}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 13.234972677595628,
   'duration': 4.706}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 6.699453551912568,
   'duration': 5.46}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.083788706739526,
   'duration': 4.136}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.3205828779599273,
   'duration': 4.72}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 6.0783242258652095,
   'duration': 5.018}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 5.5136612021857925,
   'duration': 6.511}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 4.100182149362477,
   'duration': 3.914}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 14.21311475409836,
   'duration': 5.0}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.8907103825136613,
   'duration': 5.142}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.083788706739526,
   'duration': 5.614}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 15.200364298724955,
   'duration': 6.78}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.9836065573770492,
   'duration': 3.907}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 8.109289617486338,
   'duration': 3.189}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 4.5136612021857925,
   'duration': 4.153}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 4.981785063752277,
   'duration': 4.344}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 14.21311475409836,
   'duration': 6.949}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 5.5136612021857925,
   'duration': 4.711}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 3.724954462659381,
   'duration': 3.724}]]


In [33]:
# Define colors for each algorithm
colors = {
    'dijkstra': 'blue',
    'dijkstra_binary_duration': 'red'
}

fig = go.Figure()

for sublist in data:
    for item in sublist:
        algorithm = item['algorithm']
        avg_outgoing_edges = item['average_outgoing_edges']
        duration = item['duration']
        fig.add_trace(go.Scatter(x=[avg_outgoing_edges], y=[duration], mode='markers', marker=dict(color=colors[algorithm]), name=algorithm))

fig.update_layout(title='Algorithm Performance',
                  xaxis_title='Average Outgoing Edges',
                  yaxis_title='Duration')

fig.show()


In [97]:
from numba import njit
from numba.typed import Dict
from numba.typed import List


@njit
def bisect_left(arr, x):
    lo = 0
    hi = len(arr)
    while lo < hi:
        mid = (lo + hi) // 2
        if arr[mid] < x:
            lo = mid + 1
        else:
            hi = mid
    return lo


@njit
def get_positions_fractional_cascading(m_arr, pointers, reachable_nodes, x, node):
    #locations = {}
    if pointers:
        loc, next_loc = pointers[0][bisect_left(m_arr[0], x)]
        #locations[reachable_nodes[node][0]] = loc
        for i in range(1, len(m_arr)):
            if x <= m_arr[i][next_loc - 1]:
                loc, next_loc = pointers[i][next_loc - 1]
            else:
                loc, next_loc = pointers[i][next_loc]
            #locations[reachable_nodes[node][i]] = loc
    return 0


In [99]:
get_positions_fractional_cascading(typed_m_arr_fractional, typed_pointers, tg.reachable_nodes[232], 1481541904, 229)

In [71]:
get_positions_fractional_cascading(tg.m_arr_fractional[232], tg.pointers[232], tg.reachable_nodes[232], 1481541904, 229)

In [17]:
for algorithm in algorithms:
    print(algorithm, np.mean(duration[algorithm]), np.median(duration[algorithm]), np.std(duration[algorithm]))

In [38]:
for algorithm in algorithms:
    print(algorithm, np.mean(duration[algorithm]), np.median(duration[algorithm]), np.std(duration[algorithm]))

# Results

In [ ]:
for algorithm in algorithms:
    print(algorithm, np.mean(duration[algorithm]), np.median(duration[algorithm]), np.std(duration[algorithm]))